In [2]:
!pip install symspellpy

In [3]:
import os
import math
import random
import torch
import torch.nn as nn
from torch import optim
import matplotlib.pylab as plt
from tqdm import tqdm
import cv2
import numpy as np
import torchvision
import string
from torchvision import transforms
from datetime import datetime
import csv
import pandas as pd
from glob import glob
import symspellpy
from collections import Counter

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!mkdir './HTR_dataset'
!mkdir './HTR_dataset/test_journal'
!cp '/content/drive/MyDrive/JOURNAL_HCR_PROJECT/HTR_dataset/train.tsv' './HTR_dataset/train.tsv'
!cp '/content/drive/MyDrive/JOURNAL_HCR_PROJECT/HTR_dataset/test.tsv' './HTR_dataset/test.tsv'
!cp '/content/drive/MyDrive/JOURNAL_HCR_PROJECT/HTR_dataset/train_filtered.csv' './HTR_dataset/train_filtered.csv'
!cp '/content/drive/MyDrive/JOURNAL_HCR_PROJECT/HTR_dataset/chars_code.csv' './HTR_dataset/chars_code.csv'
!cp '/content/drive/MyDrive/JOURNAL_HCR_PROJECT/HTR_dataset/train.zip' './HTR_dataset/train.zip'
!cp '/content/drive/MyDrive/JOURNAL_HCR_PROJECT/HTR_dataset/test.zip' './HTR_dataset/test.zip'
!cp '/content/drive/MyDrive/JOURNAL_HCR_PROJECT/HTR_dataset/source.txt' './HTR_dataset/gen_source.txt'
!cp '/content/drive/MyDrive/JOURNAL_HCR_PROJECT/HTR_dataset/source_valid.txt' './HTR_dataset/gen_source_valid.txt'
!cp '/content/drive/MyDrive/JOURNAL_HCR_PROJECT/Names_test_dataset.zip' './HTR_dataset/test_journal.zip'
!cp '/content/drive/MyDrive/JOURNAL_HCR_PROJECT/Grades Dataset.zip' './HTR_dataset/grades_data.zip'
!unzip './HTR_dataset/train.zip' -d './HTR_dataset'
!unzip './HTR_dataset/test.zip' -d './HTR_dataset'
!unzip './HTR_dataset/test_journal.zip' -d './HTR_dataset'
!unzip './HTR_dataset/grades_data.zip' -d './HTR_dataset'

Streaming output truncated to the last 5000 lines.
 extracting: ./HTR_dataset/train/vb219.png  
  inflating: ./HTR_dataset/train/az7086.png  
  inflating: ./HTR_dataset/train/auu853.png  
  inflating: ./HTR_dataset/train/rc1056.png  
  inflating: ./HTR_dataset/train/rc4170.png  
  inflating: ./HTR_dataset/train/as2845.png  
  inflating: ./HTR_dataset/train/rc777.png  
  inflating: ./HTR_dataset/train/iy171.png  
 extracting: ./HTR_dataset/train/as5611.png  
  inflating: ./HTR_dataset/train/ao10768.png  
  inflating: ./HTR_dataset/train/ahh298.png  
  inflating: ./HTR_dataset/train/rc1491.png  
  inflating: ./HTR_dataset/train/ap64.png  
  inflating: ./HTR_dataset/train/ahh340.png  
 extracting: ./HTR_dataset/train/aq66.png  
 extracting: ./HTR_dataset/train/aq2481.png  
  inflating: ./HTR_dataset/train/ab2108.png  
  inflating: ./HTR_dataset/train/ab6418.png  
  inflating: ./HTR_dataset/train/rc1835.png  
  inflating: ./HTR_dataset/train/as15416.png  
  inflating: ./HTR_dataset/train/a

In [4]:
alphabet = ['PAD', 'SOS', ' ', '!', '"', '%', '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', '«', '»', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Э', 'Ю', 'Я', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', 'EOS']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

char2idx = {char: idx for idx, char in enumerate(alphabet)}
idx2char = {idx: char for idx, char in enumerate(alphabet)}

In [5]:
!pip install editdistance
import editdistance

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# text to array of indicies
def text_to_labels(s, char2idx):
    return [char2idx['SOS']] + [char2idx[i] for i in s if i in char2idx.keys()] + [char2idx['EOS']]

# convert images and labels into defined data structures
def process_data(image_dir, labels_dir, ignore=[], spliter = '\t'):
    """
    params
    ---
    image_dir : str
      path to directory with images
    labels_dir : str
      path to tsv file with labels
    returns
    ---
    img2label : dict
      keys are names of images and values are correspondent labels
    chars : list
      all unique chars used in data
    all_labels : list
    """

    chars = []
    img2label = dict()

    raw = open(labels_dir, 'r', encoding='utf-8').read()
    temp = raw.split('\n')
    for t in temp:
        try:
            x = t.split(spliter)
            flag = False
            for item in ignore:
                if item in x[1]:
                    flag = True
            if flag == False:
                img2label[image_dir + x[0]] = x[1]
                for char in x[1]:
                    if char not in chars:
                        chars.append(char)
        except:
            print('ValueError:', x)
            pass

    all_labels = sorted(list(set(list(img2label.values()))))
    chars.sort()
    chars = ['PAD', 'SOS'] + chars + ['EOS']

    return img2label, chars, all_labels

# MAKE TEXT TO BE THE SAME LENGTH
class TextCollate():
    def __call__(self, batch):
        x_padded = []
        max_y_len = max([i[1].size(0) for i in batch])
        y_padded = torch.LongTensor(max_y_len, len(batch))
        y_padded.zero_()

        for i in range(len(batch)):
            x_padded.append(batch[i][0].unsqueeze(0))
            y = batch[i][1]
            y_padded[:y.size(0), i] = y
        x_padded = torch.cat(x_padded)
        return x_padded, y_padded


# TRANSLATE INDICIES TO TEXT
def labels_to_text(s, idx2char):
    """
    params
    ---
    idx2char : dict
        keys : int
            indicies of characters
        values : str
            characters
    returns
    ---
    S : str
    """
    S = "".join([idx2char[i] for i in s])
    if S.find('EOS') == -1:  
        return S
    else:
        return S[:S.find('EOS')] 


# COMPUTE CHARACTER ERROR RATE
def char_error_rate(p_seq1, p_seq2):
    """
    params
    ---
    p_seq1 : str
    p_seq2 : str
    returns
    ---
    cer : float
    """
    p_vocab = set(p_seq1 + p_seq2)
    p2c = dict(zip(p_vocab, range(len(p_vocab)))) 
    c_seq1 = [chr(p2c[p]) for p in p_seq1]
    c_seq2 = [chr(p2c[p]) for p in p_seq2]
    return editdistance.eval(''.join(c_seq1),
                             ''.join(c_seq2)) / max(len(c_seq1), len(c_seq2))


def process_image(img):
    """
    params:
    ---
    img : np.array
    returns
    ---
    img : np.array
    """
    w, h, _ = img.shape
    new_w = 64
    new_h = int(h * (new_w / w))
    img = cv2.resize(img, (new_h, new_w))
    w, h, _ = img.shape

    img = img.astype('float32')

    new_h = 256
    if h < new_h:
        add_zeros = np.full((w, new_h - h, 3), 255)
        img = np.concatenate((img, add_zeros), axis=1)

    if h > new_h:
        img = cv2.resize(img, (new_h, new_w))

    return img



# GENERATE IMAGES FROM FOLDER
def generate_data(img_paths):
    """
    params
    ---
    names : list of str
        paths to images
    returns
    ---
    data_images : list of np.array
        images in np.array format
    """
    data_images = []
    for path in img_paths:
        img = cv2.imread(path)
        try:
            img = process_image(img)
            data_images.append(img.astype('uint8'))
        except:
            img = process_image(img)
    return data_images

def train_valid_split(img2label, val_part=0.3):
    """
    params
    ---
    img2label : dict
        keys are paths to images, values are labels (transcripts of crops)
    returns
    ---
    imgs_val : list of str
        paths
    labels_val : list of str
        labels
    imgs_train : list of str
        paths
    labels_train : list of str
        labels
    """

    imgs_val, labels_val = [], []
    imgs_train, labels_train = [], []

    N = int(len(img2label) * val_part)
    items = list(img2label.items())
    random.shuffle(items)
    for i, item in enumerate(items):
        if i < N:
            imgs_val.append(item[0])
            labels_val.append(item[1])
        else:
            imgs_train.append(item[0])
            labels_train.append(item[1])
    print('valid part:{}'.format(len(imgs_val)))
    print('train part:{}'.format(len(imgs_train)))
    return imgs_val, labels_val, imgs_train, labels_train

def validate(model, dataset, char2idx, idx2char):
    model.eval()
    show_count = 0
    wer_overall = 0
    cer_overall = 0
    with torch.no_grad():
        for (src, trg) in tqdm(dataset):
            src = src.unsqueeze(0)
            img = np.moveaxis(src[0].numpy(), 0, 2)
            if torch.cuda.is_available():
              src = src.cuda()

            out_indexes = [char2idx['SOS'], ]

            for i in range(100):
                
                trg_tensor = torch.LongTensor(out_indexes).unsqueeze(1).to(device)
                output = model(src,trg_tensor)
                out_token = output.argmax(2)[-1].item()
                out_indexes.append(out_token)
                if out_token == char2idx['EOS']:
                    break

            out_char = labels_to_text(out_indexes[1:], idx2char)
            real_char = labels_to_text(trg[1:].numpy(), idx2char)
            output_corrected = correct(out_char)
            wer_overall += int(real_char != output_corrected)
            if output_corrected:
                cer = char_error_rate(real_char, output_corrected)
            else:
                cer = 1
            
            cer_overall += cer
    
    return cer_overall / len(dataset) * 100, wer_overall / len(dataset) * 100

def evaluate(model, criterion, iterator):
      """
      params
      ---
      model : nn.Module
      criterion : nn.Object
      iterator : torch.utils.data.DataLoader
      returns
      ---
      epoch_loss / len(iterator) : float
          overall loss
      """
      model.eval()
      epoch_loss = 0
      with torch.no_grad():
          for (src, trg) in tqdm(iterator):
              src, trg = src.cuda(), trg.cuda()
              output = model(src, trg[:-1, :])
              loss = criterion(output.view(-1, output.shape[-1]), torch.reshape(trg[1:, :], (-1,)))
              epoch_loss += loss.item()
      return epoch_loss / len(iterator)

def prediction(model, test_dir, img2label, char2idx, idx2char, show_result = False):
    preds = {}
    model.eval()
    with torch.no_grad():
        preds = {}
        for img_path, target_str in tqdm(img2label.items()):
            img = generate_data([img_path])[0]
            transform = transforms.Compose([
              #transforms.ToPILImage(),
              transforms.ToTensor(),
              transforms.Normalize((0.5,), (0.5,))])
            norm_img = transform(img)

            src = torch.FloatTensor(norm_img).unsqueeze(0).to(device)
            out_indexes = [char2idx['SOS'], ]

            for i in range(100):
                trg_tensor = torch.LongTensor(out_indexes).unsqueeze(1).to(device)
                output = model(src,trg_tensor)
                out_token = output.argmax(2)[-1].item()
                out_indexes.append(out_token)
                if out_token == char2idx['EOS']:
                    break

            pred = labels_to_text(out_indexes[1:], idx2char)
            preds[img_path] = pred

            if show_result:
                print("File name: {} Predict: {} True: {}".format(img_path, pred, target_str))
                plt.imshow(img)
                plt.show()
        
        return preds

In [6]:
from torchvision import models
class TransformerModel(nn.Module):
    def __init__(self, bb_name, outtoken, hidden, enc_layers=1, dec_layers=1, nhead=1, dropout=0.1, pretrained=False):
        super(TransformerModel, self).__init__()
        self.backbone = models.__getattribute__(bb_name)(pretrained=pretrained)
        self.backbone.fc = nn.Conv2d(2048, int(hidden/2), 1)

        self.pos_encoder = PositionalEncoding(hidden, dropout)
        self.decoder = nn.Embedding(outtoken, hidden)
        self.pos_decoder = PositionalEncoding(hidden, dropout)
        self.transformer = nn.Transformer(d_model=hidden, nhead=nhead, num_encoder_layers=enc_layers,
                                          num_decoder_layers=dec_layers, dim_feedforward=hidden * 4, dropout=dropout,
                                          activation='relu')

        self.fc_out = nn.Linear(hidden, outtoken)
        self.src_mask = None
        self.trg_mask = None
        self.memory_mask = None
        
        print('backbone: {}'.format(type(self.backbone)))
        print('layers: {}'.format(enc_layers))
        print('heads: {}'.format(nhead))
        print('dropout: {}'.format(dropout))
        print(f'{count_parameters(self):,} trainable parameters')

    def generate_square_subsequent_mask(self, sz):
        mask = torch.triu(torch.ones(sz, sz), 1)
        mask = mask.masked_fill(mask == 1, float('-inf'))
        return mask

    def make_len_mask(self, inp):
        return (inp == 0).transpose(0, 1)

    def forward(self, src, trg):
        '''
        params
        ---
        src : Tensor [64, 3, 64, 256] : [B,C,H,W]
            B - batch, C - channel, H - height, W - width
        trg : Tensor [13, 64] : [L,B]
            L - max length of label
        '''
        if self.trg_mask is None or self.trg_mask.size(0) != len(trg):
            self.trg_mask = self.generate_square_subsequent_mask(len(trg)).to(trg.device) 
        x = self.backbone.conv1(src)

        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)
        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x) # [64, 2048, 2, 8] : [B,C,H,W]
            
        x = self.backbone.fc(x) # [64, 256, 2, 8] : [B,C,H,W]
        x = x.permute(0, 3, 1, 2) # [64, 8, 256, 2] : [B,W,C,H]
        x = x.flatten(2) # [64, 8, 512] : [B,W,CH]
        x = x.permute(1, 0, 2) # [8, 64, 512] : [W,B,CH]
        
        src_pad_mask = self.make_len_mask(x[:, :, 0])
        src = self.pos_encoder(x) # [8, 64, 512]
        trg_pad_mask = self.make_len_mask(trg)
        trg = self.decoder(trg)
        trg = self.pos_decoder(trg)

        output = self.transformer(src, trg, src_mask=self.src_mask, tgt_mask=self.trg_mask,
                                  memory_mask=self.memory_mask,
                                  src_key_padding_mask=src_pad_mask, tgt_key_padding_mask=trg_pad_mask,
                                  memory_key_padding_mask=src_pad_mask) # [13, 64, 512] : [L,B,CH]
        output = self.fc_out(output) # [13, 64, 92] : [L,B,H]

        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.scale = nn.Parameter(torch.ones(1))

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(
            0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.scale * self.pe[:x.size(0), :]
        return self.dropout(x) 

In [7]:
# store list of images' names (in directory) and does some operations with images
class TextLoader(torch.utils.data.Dataset):
    def __init__(self, images_name, labels, char2idx, idx2char, eval=False):
        """
        params
        ---
        images_name : list
            list of names of images (paths to images)
        labels : list
            list of labels to correspondent images from images_name list
        char2idx : dict
        idx2char : dict
        """
        self.images_name = images_name
        self.labels = labels
        self.char2idx = char2idx
        self.idx2char = idx2char
        self.eval = eval
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            #transforms.Resize((int(hp.height *1.05), int(hp.width *1.05))),
            #transforms.RandomCrop((hp.height, hp.width)),
            # transforms.ColorJitter(contrast=(0.5,1),saturation=(0.5,1)),
            # transforms.RandomRotation(degrees=(-9, 9), fill=255),
            # transforms.RandomAffine(10 ,None ,[0.6 ,1] ,3 ,fillcolor=255),
            # transforms.transforms.GaussianBlur(3, sigma=(0.1, 1.9)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

    def _transform(self, X):
        return self.transform(X)
        
    def __getitem__(self, index):
        img = self.images_name[index]
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transform(img)
        #img = np.transpose(img, (2, 0, 1))


        label = text_to_labels(self.labels[index], self.char2idx)
        return (torch.FloatTensor(img), torch.LongTensor(label))

    def __len__(self):
        return len(self.labels)

In [8]:
model = TransformerModel('resnet50', len(alphabet), hidden=512, enc_layers=2, dec_layers=2, nhead=4, dropout=0.0).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/JOURNAL_HCR_PROJECT/training_results/2022-04-12_11:21:49/htr_transformer.pth'))

backbone: <class 'torchvision.models.resnet.ResNet'>
layers: 2
heads: 4
dropout: 0.0
38,841,758 trainable parameters


<All keys matched successfully>

## Load names string

In [10]:
!pip install russian-names

     |████████████████████████████████| 652 kB 12.9 MB/s 


In [9]:
from russian_names import RussianNames

In [44]:
rn = ()
for i in tqdm(range(2000)):
  rn += RussianNames(count=5000, gender=0.5, surname_max_len=15, uppercase=False, patronymic = False).get_batch()
  rn += RussianNames(count=2500, gender=0.5, name_reduction = True, surname_max_len=15, uppercase=False, patronymic = False).get_batch()

100%|██████████| 2000/2000 [25:52<00:00,  1.29it/s]


In [50]:
symspell = symspellpy.SymSpell()

In [51]:
dicts = [name for person in rn for name in person.split(' ')]
name_freq = Counter(dicts)

In [52]:
with open('dictionary.txt', 'w') as f:
    for name, freq in name_freq.items():
        f.write('{} {}\n'.format(name, freq))

In [53]:
symspell.load_dictionary('dictionary.txt', term_index=0, count_index=1)

True

In [54]:
def correct(s):
    def correct_word(w):
        tmp = symspell.lookup(w, symspellpy.Verbosity.CLOSEST)
        if len(tmp):
            return tmp[0].term
        else:
            return w

    return ' '.join([correct_word(word) for word in s.split(' ')])

In [55]:
%%time
correct('Сеифанва А.')

CPU times: user 434 µs, sys: 6 µs, total: 440 µs
Wall time: 446 µs


'Селифанова А.'

##Проверка работы на модели

In [56]:
img2label, _, _ = process_data('/content/HTR_dataset/Names_test_dataset/', '/content/HTR_dataset/Names_test_dataset/file_target_mapping.txt', spliter = ',')
_, _, X_train, y_train = train_valid_split(img2label,val_part=0)
X_train = generate_data(X_train)
names_dataset = TextLoader(X_train, y_train, char2idx ,idx2char, eval=False)

ValueError: ['']
valid part:0
train part:403


In [57]:
TRAIN_DATA_PATH = '/content/HTR_dataset/Grades Dataset/Train'

def change_target(target_idx):
    labels_mapp = ['2',  '3',  '4',  '5',  '',  'н']
    label = text_to_labels(labels_mapp[target_idx], char2idx)
    return torch.LongTensor(label)

transform = transforms.Compose([
                transforms.Lambda(lambda x: np.asarray(x.convert('RGB'))),
                transforms.Lambda(process_image),
                #transforms.Resize((64, 256)),
                transforms.Lambda(lambda x: x.astype('uint8')),
                transforms.ToPILImage(),
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,))])

grades_dataset = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=transform, target_transform=change_target)

In [58]:
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, names_dataset, grades_dataset):
        self.names_dataset = names_dataset
        self.grades_dataset = grades_dataset
        self.shuffled_grades = self.regenerate_grades_data()
    
    def regenerate_grades_data(self):
        random_idxs = random.sample(range(1, len(self.grades_dataset)), int(len(self.grades_dataset)*0.7))
        shuffled_grades = []
        for i in random_idxs:
            shuffled_grades.append(self.grades_dataset[i])

        return shuffled_grades

    def __getitem__(self, i):
        if i < len(self.names_dataset):
            return self.names_dataset[i][0], self.names_dataset[i][1]
        else:
            names_data_len  = len(self.names_dataset)
            return self.shuffled_grades[i - names_data_len]

    def __len__(self):
        lenght = len(self.names_dataset) + len(self.shuffled_grades)
        return lenght

mix_journal_dataset = ConcatDataset(
                 names_dataset,
                 grades_dataset
             )
             
mix_journal_dataset_train, mix_journal_dataset_val = torch.utils.data.random_split(mix_journal_dataset, [int(len(mix_journal_dataset)*0.8), int(len(mix_journal_dataset)*0.2)])

mix_journal_loader_train = torch.utils.data.DataLoader(
             mix_journal_dataset_train,
             batch_size=64, shuffle=True,
             num_workers=2, collate_fn = TextCollate())

mix_journal_loader_val = torch.utils.data.DataLoader(
             mix_journal_dataset_val,
             batch_size=8, shuffle=True,
             num_workers=2, collate_fn = TextCollate())

In [59]:
img2label, _, _ = process_data('/content/HTR_dataset/Names_test_dataset/test/', '/content/HTR_dataset/Names_test_dataset/test/test_target_mapping.txt', spliter = ',')
_, _, X_test, y_test = train_valid_split(img2label,val_part=0)
X_test = generate_data(X_test)
names_dataset_test = TextLoader(X_test, y_test, char2idx ,idx2char, eval=False)

valid part:0
train part:42


In [61]:
cer, wer = validate(model, names_dataset_test, char2idx, idx2char)
print(cer, wer)

100%|██████████| 42/42 [00:24<00:00,  1.69it/s]

11.145614811581199 54.761904761904766


In [62]:
def test(model, test_dir, img_target_mapping_path, char2idx, idx2char, spliter, print_results = True, case=True, punct=False):
    img2label = dict()
    raw = open(img_target_mapping_path, 'r', encoding='utf-8').read()
    temp = raw.split('\n')
    for t in temp:
        if not spliter:
            x = t.split('\t')
        else: 
            x = t.split(spliter)
        img2label[os.path.join(test_dir, x[0])] = x[1]
    
    preds = prediction(model, test_dir, img2label, char2idx, idx2char)        
    N = len(preds)

    wer = 0
    cer = 0

    for item in preds.items():
        img_name = item[0]
        true_trans = img2label[os.path.join(test_dir, img_name)]
        predicted_trans = item[1]
        predicted_trans = correct(predicted_trans)

        if 'ё' in true_trans:
            true_trans = true_trans.replace('ё', 'е')
        if 'ё' in predicted_trans:
            predicted_trans = predicted_trans.replace('ё', 'е')

        if not case:
            true_trans = true_trans.lower()
            predicted_trans = predicted_trans.lower()

        if not punct:
            true_trans = true_trans.translate(str.maketrans('', '', string.punctuation))
            predicted_trans = predicted_trans.translate(str.maketrans('', '', string.punctuation))

        if true_trans != predicted_trans and print_results:
            print('true:', true_trans)
            print('predicted:', predicted_trans)
            print('cer:', char_error_rate(predicted_trans, true_trans))
            print('---')
            wer += 1
            cer += char_error_rate(predicted_trans, true_trans)

    character_accuracy = 1 - cer / N
    string_accuracy = 1 - (wer / N)
    return character_accuracy, string_accuracy

In [63]:
word_accur, char_accur = test(model, '/content/HTR_dataset/Names_test_dataset/test/', '/content/HTR_dataset/Names_test_dataset/test/test_target_mapping.txt', 
                              char2idx, idx2char, spliter = ',', print_results = True, 
                              case=True, punct=True)
print(word_accur, char_accur)

100%|██████████| 42/42 [00:24<00:00,  1.70it/s]

true: Верещагина Е.
predicted: Веремагина Е.
cer: 0.07692307692307693
---
true: Ларионов А.
predicted: Гаринов А.
cer: 0.18181818181818182
---
true: Макеев Егор
predicted: Макееев Игорь
cer: 0.23076923076923078
---
true: Высоцкий М.
predicted: Вырский М.
cer: 0.2727272727272727
---
true: Полякова Олив.
predicted: Болякова Алиса
cer: 0.2857142857142857
---
true: Чихова К.
predicted: Чигова К.
cer: 0.1111111111111111
---
true: Мухин Д.
predicted: Мухин Я.
cer: 0.125
---
true: Селиванова А.
predicted: Селифанова А.
cer: 0.07692307692307693
---
true: Лыков Матвей
predicted: Лыков Матяев
cer: 0.16666666666666666
---
true: Костин Марк
predicted: Костин Марков
cer: 0.15384615384615385
---
true: Головин И.
predicted: Половин И.
cer: 0.1
---
true: Покровская К.
predicted: Лаковская К.
cer: 0.23076923076923078
---
true: Суслова В.
predicted: Салова В.
cer: 0.2
---
true: Соколов Степан
predicted: Соколов Стаин
cer: 0.21428571428571427
---
true: Макарова Ксен.
predicted: Макарова Ксел.
cer: 0.0714